<a href="https://colab.research.google.com/github/Ana-AlonsoCanizares/deeplearning/blob/main/Pr%C3%A1ctica2_DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 2: Clasificar texto con Codificador Transformer

**Integrantes del grupo**

Álvaro García Cid

Ana Alonso Cañizares

Álvaro García Parra

## Objetivo

Utilizar un Codificador Transformer para clasificar las noticias de Reuters en 46 temas mutuamente excluyentes. Como cada noticia debe clasificarse en una sola categoría, es un problema de "clasificación multiclase de una sola etiqueta".

El Reuters dataset es un conjunto de noticias breves y sus temas, publicado por Reuters en 1986. Son 46 temas diferentes; algunos temas están más representados que otros, pero cada uno tiene, al menos, 10 ejemplos en el conjunto de entrenamiento.

Al igual que IMDB y MNIST, el conjunto de datos de Reuters viene empaquetado como parte de Keras.

## Desarrollo

### Importación de librerías, funciones y variables

In [18]:
# Importación de las librerías a utilizar
from keras.datasets import reuters
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import kerastuner as kt

<ipython-input-18-771e1265936d>:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


### Lectura de datos

In [2]:
vocab_size = 10000  # Tamaño del vocabulario
batch_size = 32     # Tamaño del lote
embed_dim = 256     # Dimensión del embedding
num_heads = 2       # Número de cabezas del MultiHead
dense_dim = 32      # Nº de neuronas de la capa densa
EPOCHS = 20         # Nº de épocas

In [3]:
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=vocab_size)

# Dividir los datos en conjuntos de entrenamiento y validación
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

2110848/2110848 [==============================] - 0s 0us/step


In [4]:
# Calcula la longitud de cada fila y luego toma la media como la longitud máxima
lengths = [len(text) for text in train_data]
max_length = round(sum(lengths) / len(lengths)) # Tamaño máximo de tokens

# Aplicar relleno o truncado a las secuencias
train_data = tf.keras.preprocessing.sequence.pad_sequences(train_data, maxlen=max_length, padding='post')
val_data = tf.keras.preprocessing.sequence.pad_sequences(val_data, maxlen=max_length, padding='post')
test_data = tf.keras.preprocessing.sequence.pad_sequences(test_data, maxlen=max_length, padding='post')

In [5]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        # Tamaño de los vectores de los tokens de entrada
        self.embed_dim = embed_dim
        # Tamaño de la capa densa interna
        self.dense_dim = dense_dim
        # Número de attention heads
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    # El cálculo va en call()
    def call(self, inputs, mask=None):
        # La máscara que generará la capa Embedding
        # será 2D, pero la capa de atención espera
        # ser 3D o 4D, por lo que ampliamos su rango
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    # Implementamos la serialización para
    # que podamos guardar el modelo
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [6]:
class PositionalEmbedding(layers.Layer):
    # Una desventaja de las incrustaciones de posición es que
    # la longitud de la secuencia debe conocerse de antemano
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        # Prepara una capa de embedding para los índices de token.
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            # Y otro para las posiciones te tokens
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        # Agrega ambos vectores embeddings juntos
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        # Al igual que la capa de embedding,
        # esta capa debería poder generar una
        # máscara para que podamos ignorar los
        # ceros de relleno en las entradas.
        # El framework llamará automáticamente
        # al método compute_mask y la máscara
        # se propagará a la siguiente capa.
        return tf.math.not_equal(inputs, 0)

    # Implementamos la serialización para que
    # podamos guardar el modelo.
    def get_config(self):
        config = super().get_config()
        # config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [7]:
# Creamos los Dataset
# Datos de entrenamiento
train_ds = tf.data.Dataset.from_tensor_slices((list(train_data), list(train_labels)))
train_ds = train_ds.batch(batch_size)
train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)  # Precargamos datos para acelerar el entrenamiento

# Datos de validación
val_ds = tf.data.Dataset.from_tensor_slices((list(val_data), list(val_labels)))
val_ds = val_ds.batch(batch_size)
val_ds = val_ds.prefetch(tf.data.experimental.AUTOTUNE)  # Precargamos datos para acelerar el entrenamiento

# Datos de evaluación
test_ds = tf.data.Dataset.from_tensor_slices((list(test_data), list(test_labels)))
test_ds = test_ds.batch(batch_size)
test_ds = test_ds.prefetch(tf.data.experimental.AUTOTUNE)  # Precargamos datos para acelerar el entrenamiento

In [8]:
inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = PositionalEmbedding(max_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(46, activation="softmax")(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 positional_embedding (Posi  (None, None, 256)         2597376   
 tionalEmbedding)                                                
                                                                 
 transformer_encoder (Trans  (None, None, 256)         543776    
 formerEncoder)                                                  
                                                                 
 global_max_pooling1d (Glob  (None, 256)               0         
 alMaxPooling1D)                                                 
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                             

In [9]:
model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)
print(f"Test acc: {model.evaluate(test_ds)[1]:.3f}")

Epoch 1/20
225/225 [==============================] - 26s 62ms/step - loss: 2.2634 - accuracy: 0.5084 - val_loss: 1.4188 - val_accuracy: 0.6800
Epoch 2/20
225/225 [==============================] - 4s 16ms/step - loss: 1.3639 - accuracy: 0.6848 - val_loss: 1.1289 - val_accuracy: 0.7340
Epoch 3/20
225/225 [==============================] - 3s 15ms/step - loss: 1.0344 - accuracy: 0.7626 - val_loss: 1.0001 - val_accuracy: 0.7596
Epoch 4/20
225/225 [==============================] - 3s 15ms/step - loss: 0.8207 - accuracy: 0.7985 - val_loss: 1.0094 - val_accuracy: 0.7774
Epoch 5/20
225/225 [==============================] - 4s 16ms/step - loss: 0.6771 - accuracy: 0.8344 - val_loss: 1.0416 - val_accuracy: 0.7696
Epoch 6/20
225/225 [==============================] - 3s 15ms/step - loss: 0.5701 - accuracy: 0.8611 - val_loss: 1.0161 - val_accuracy: 0.7802
Epoch 7/20
225/225 [==============================] - 3s 15ms/step - loss: 0.4727 - accuracy: 0.8809 - val_loss: 0.9773 - val_accuracy: 0.788

In [10]:
inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = PositionalEmbedding(max_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(46, activation="softmax")(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer="Adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 positional_embedding_1 (Po  (None, None, 256)         2597376   
 sitionalEmbedding)                                              
                                                                 
 transformer_encoder_1 (Tra  (None, None, 256)         543776    
 nsformerEncoder)                                                
                                                                 
 global_max_pooling1d_1 (Gl  (None, 256)               0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                           

In [11]:
model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)
print(f"Test acc: {model.evaluate(test_ds)[1]:.3f}")

Epoch 1/20
225/225 [==============================] - 24s 89ms/step - loss: 2.3411 - accuracy: 0.4910 - val_loss: 1.4054 - val_accuracy: 0.6923
Epoch 2/20
225/225 [==============================] - 4s 17ms/step - loss: 1.2517 - accuracy: 0.7203 - val_loss: 1.0224 - val_accuracy: 0.7702
Epoch 3/20
225/225 [==============================] - 4s 17ms/step - loss: 0.7804 - accuracy: 0.8117 - val_loss: 0.9971 - val_accuracy: 0.7902
Epoch 4/20
225/225 [==============================] - 4s 16ms/step - loss: 0.5315 - accuracy: 0.8661 - val_loss: 1.0543 - val_accuracy: 0.7774
Epoch 5/20
225/225 [==============================] - 4s 16ms/step - loss: 0.3699 - accuracy: 0.9044 - val_loss: 0.9815 - val_accuracy: 0.8002
Epoch 6/20
225/225 [==============================] - 4s 17ms/step - loss: 0.2645 - accuracy: 0.9303 - val_loss: 1.1139 - val_accuracy: 0.7980
Epoch 7/20
225/225 [==============================] - 4s 17ms/step - loss: 0.2069 - accuracy: 0.9422 - val_loss: 1.3119 - val_accuracy: 0.784

##PRUEBA TUNER OPTIMIZACIÓN BAYESIANA

In [17]:
pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.4 MB/s eta 0:00:00


In [19]:
def build_model(hp):
    vocab_size = 10000  # Tamaño fijo del vocabulario
    max_length = hp.Int('max_length', min_value=100, max_value=500, step=50)
    num_heads = hp.Int('num_heads', min_value=2, max_value=8, step=2)
    embed_dim = hp.Int('embed_dim', min_value=64, max_value=512, step=64)
    dense_dim = hp.Int('dense_dim', min_value=32, max_value=128, step=32)

    inputs = keras.Input(shape=(None,), dtype="int64")
    x = layers.Embedding(vocab_size, embed_dim)(inputs)
    x = PositionalEmbedding(max_length, vocab_size, embed_dim)(inputs)
    x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(46, activation="softmax")(x)

    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])

    return model

# Definimos el tuner
tuner = kt.BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Número de combinaciones de hiperparámetros a probar
    executions_per_trial=1,  # Cuántas veces se ejecutará cada modelo para promediar métricas
    directory='my_dir',  # Directorio donde se guardan los logs
    project_name='transformer_tuning'
)

# Callback para detener temprano si no se observan mejoras
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Inicia la búsqueda
tuner.search(train_ds, epochs=10, validation_data=val_ds, callbacks=[stop_early])

# Obtén el mejor modelo
best_model = tuner.get_best_models(num_models=1)[0]

# Evalúa el mejor modelo
loss, accuracy = best_model.evaluate(test_ds)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

# Sacamos la mejor combinación lograda
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
El mejor número de cabezas de atención: {best_hps.get('num_heads')}
El mejor tamaño de embedding: {best_hps.get('embed_dim')}
La mejor dimensión de la capa densa: {best_hps.get('dense_dim')}
El mejor tamaño máximo de secuencia: {best_hps.get('max_length')}
""")

Trial 10 Complete [00h 01m 37s]
val_accuracy: 0.7885364294052124

Best val_accuracy So Far: 0.8007791042327881
Total elapsed time: 00h 18m 45s
71/71 [==============================] - 1s 10ms/step - loss: 1.1698 - accuracy: 0.7685
Test Loss: 1.1698168516159058, Test Accuracy: 0.7684773206710815

El mejor número de cabezas de atención: 6
El mejor tamaño de embedding: 192
La mejor dimensión de la capa densa: 128
El mejor tamaño máximo de secuencia: 100

